<a href="https://colab.research.google.com/github/ClaireZixiWang/robustness-interpretability/blob/main/Resnet18_naturally_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
from torch.optim import SGD
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models import resnet18, resnet50
import pandas as pd
import numpy as np
import time
from torch.optim.lr_scheduler import ExponentialLR

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
PATH = '/content/drive/MyDrive/resnet18/'

In [6]:
# network parameters
BATCH_SIZE = 128
EPOCHS = 350
LEARNING_RATE = 0.01

In [7]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False,
                                          num_workers=4)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
total_step = len(train_loader)

In [9]:
model = resnet18(pretrained=False)
model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=1/3)

In [ ]:
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # epoch, step/epoch, total_step, loss
        if (i%100==1):
            print ("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                   .format(epoch, EPOCHS, i, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        scheduler.step()

    if epoch % 50==0:
      checkpoint_name = 'resnet_{:04d}.pth'.format(epoch)
      torch.save(model.state_dict(), PATH+checkpoint_name)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [0/350], Step [1/391], Loss: 0.3945
Epoch [0/350], Step [101/391], Loss: 0.5284
Epoch [0/350], Step [201/391], Loss: 0.6425
Epoch [0/350], Step [301/391], Loss: 0.4517
Epoch [1/350], Step [1/391], Loss: 0.5246
Epoch [1/350], Step [101/391], Loss: 0.5477
Epoch [1/350], Step [201/391], Loss: 0.4592
Epoch [1/350], Step [301/391], Loss: 0.4775


In [ ]:
checkpoint_name = 'resnet_{:04d}.pth'.format(EPOCHS)
torch.save(model.state_dict(), PATH+checkpoint_name)